In [1]:
from transformers import T5Tokenizer, T5ForConditionalGeneration

from sentence_transformers import InputExample
from sentence_transformers import datasets
from sentence_transformers import models, SentenceTransformer
from sentence_transformers import losses
from sentence_transformers import SentenceTransformer, util 
import torch

from tqdm.auto import tqdm
from pathlib import Path

from my_util import get_chunks, get_topk_similarity

/Users/nezumikozo/Documents/workspace/rag-redis-demo/venv/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Synthesize queries given content

In [2]:
default_query = "what are scope 1 emissions?"

In [3]:
content, metadata = get_chunks(company_name="novo_nordisk")

In [17]:
def synthesize_queries(paragraphs, num_queries=3, debug=False):
    tokenizer = T5Tokenizer.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')
    model = T5ForConditionalGeneration.from_pretrained('BeIR/query-gen-msmarco-t5-large-v1')

    # Some layers in the model behave differently during training and inference.
    # To ensure the model is running in “inference mode”, we call model.eval().
    model.eval()

    pairs = []
    file_count = 0

    # set to no_grad as we don't need to calculate gradients for back prop
    with torch.no_grad():
        # loop through each passage individually
        for p in tqdm(paragraphs):
            p = p.replace('\t', ' ')
            # create input tokens
            input_ids = tokenizer.encode(p, return_tensors='pt')
            # generate output tokens (query generation)
            outputs = model.generate(
                input_ids=input_ids,
                max_length=64,
                do_sample=True,
                top_p=0.95,
                num_return_sequences=num_queries
            )
            # decode output tokens to human-readable language
            for output in outputs:
                query = tokenizer.decode(output, skip_special_tokens=True)
                # append (query, passage) pair to pairs list, separate by \t
                pairs.append(query.replace('\t', ' ') + '\t' + p)

            # once we have 200 pairs write to file
            if len(pairs) > 150:
                # save as tsv
                with open(f'data/pairs_{file_count}.tsv', 'w', encoding='utf-8') as fp:
                    fp.write('\n'.join(pairs))
                file_count += 1
                pairs = []

                if debug:
                    print(f"Paragraph:\n{p}")
                    print("\nGenerated queries:")
                    for idx, output in enumerate(outputs):
                        query = tokenizer.decode(output, skip_special_tokens=True)
                        print(f'{idx + 1}: {query}')
                    
                    

    if pairs is not None:
        # save the final, smaller than 1024 batch
        with open(f'data/pairs_{file_count}.tsv', 'w', encoding='utf-8') as fp:
            fp.write('\n'.join(pairs))

        if debug:
            print(f"Paragraph:\n{p}")
            print("\nGenerated queries:")
            for idx, output in enumerate(outputs):
                query = tokenizer.decode(output, skip_special_tokens=True)
                print(f'{idx + 1}: {query}')

In [18]:
synthesize_queries(
    paragraphs=content, 
    num_queries=3, 
    debug=True,
)

 16%|████████████████████▋                                                                                                                | 51/327 [08:48<48:24, 10.53s/it]

Paragraph:
Company: novo_nordisk. portfolio and US insulin portfolio by incorporating them into

according to the Sarbanes-Oxley Act by an independent audit

our ESG statement (please refer to note 8.6 on US pricing).

We have implemented a set of data and artificial intelligence

firm elected at the Annual General Meeting. As part of our

ethics principles in our Global Ethics and Compliance

ESG responsibility, we voluntarily include an Assurance Report

We continue working on implementing recommendations

Framework. These principles define Novo Nordisk’s ethical

from an independent external auditor for ESG reporting in

from the Taskforce on Climate-related Financial Disclosures

data management across the Group and aim to promote a

the Annual Report. The assurance provider reviews whether

(TCFD), taking a stepwise approach to incorporating material

sound and ethical data culture within Novo Nordisk and in

the consolidated ESG statement is accurately presented.

climate-related

 31%|█████████████████████████████████████████▏                                                                                          | 102/327 [18:20<41:58, 11.19s/it]

Paragraph:
Company: novo_nordisk. 650

cash value of up to DKK 28 billion. The total programme

550

may be reduced in size if significant business development

opportunities arise during 2023. Novo Nordisk expects

450

to conduct the majority of the new share repurchase

350

programme according to the safe harbour rules in MAR. At

Jan

Feb

Mar

Apr

May

Jun

Jul

Aug

Sep

Oct

Nov

Dec

the Annual General Meeting in March 2023, the Board of

Directors will propose a further reduction in the company’s

4. OMXC25 and pharmaceutical industry development have been rebased to Novo Nordisk share price in January 2022. 5. Abbvie, Amgen, AstraZeneca, Biogen Idec Inc, Bristol-Myers Squibb, Eli Lilly & Co., Gilead Sciences, Glaxo Smith Kline, Johnson & Johnson, Lundbeck, Merck & Co, Novartis AG, Pfizer, Roche and Sanofi-Aventis SA.

B share capital, corresponding to approximately 1.1% of the

total share capital, by cancelling 25 million treasury shares.

Share price development

From end

 47%|█████████████████████████████████████████████████████████████▊                                                                      | 153/327 [27:44<30:04, 10.37s/it]

...

Paragraph:
Company: novo_nordisk. Contents Introducing Novo Nordisk Strategic Aspirations Key risks Management Consolidated statements Additional information

Notes to the consolidated ESG statement

Section 6 Basis of preparation

General reporting standards and principles Novo Nordisk's annual reporting complies with the Danish Financial Statements Act. Sections 99a, 99b, 99d and 107d specify the requirements to report on the management of risks related to the environment, climate, human rights, labour and social conditions, anti-corruption, gender distribution and data ethics. These requirements are addressed in the Management review.

As recommended by the Taskforce on Climate-related Financial Disclosures (TCFD), Novo Nordisk is working to integrate two climate change scenarios into the risk management process to identify short-, medium- and long-term risks within the production and supply chain:

– Limiting temperature increase to well below 2ºC scenario, preferably 1.5ºC, compar

 94%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▌        | 306/327 [54:39<03:15,  9.31s/it]

Paragraph:
Company: novo_nordisk. Management's responsibility Management of Novo Nordisk A/S is responsible for designing, implementing, and maintaining internal controls over information relevant to the preparation of the ESG data and information in the ESG statement, ensuring they are free from material misstatement, whether due to fraud or error. Furthermore, Management is responsible for establishing objective accounting policies for the preparation of ESG data, for the overall content of the ESG statement, and for measuring and reporting ESG data in accordance with the Basis of preparation and the ESG accounting policies.

Work performed We are required to plan and perform our work in order to consider the risk of material misstatement in the ESG statement. To do so, we have: – conducted interviews with data owners and internal stakeholders to understand the key processes and control activities for measuring, recording and reporting the ESG data;

– performed limited substantive t

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 327/327 [58:38<00:00, 10.76s/it]


Paragraph:
Company: novo_nordisk. 3. Guaranties given for subsidiaries mainly relate to guaranties towards Novo Nordisk

Finance (Netherlands) B.V. related to issuance of Eurobonds.

Novo Nordisk A/S is included in the consolidated financial statements of the Novo Nordisk Foundation.

14 Fee to statutory auditors

DKK million

2022

2021

Novo Nordisk A/S and its Danish subsidiaries are jointly taxed with the Danish companies in Novo Holdings A/S. The joint taxation also covers withholding taxes in the form of dividend tax, royalty tax and interest tax. The Danish companies are jointly and severally liable for the joint taxation. Any subsequent adjustments to income taxes and withholding taxes may lead to a larger liability. The tax for the individual companies is allocated in full on the basis of the expected taxable income.

Statutory audit1

15

8

Audit-related services

Tax advisory services

2

1

2

2

For information on pending litigation and other contingencies, please refer t

## What are the generated queries for our answer?

In [20]:
synthesize_queries(
    paragraphs=[content[251]], 
    num_queries=3, 
    debug=True,
)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:32<00:00, 32.81s/it]


Paragraph:
Company: novo_nordisk. 51,951

42,138

123.3%

2020

28,565

42,138

67.8%

Contents Introducing Novo Nordisk Strategic Aspirations Key risks Management Consolidated statements Additional information

Note

2022

2021

Statement of Environmental, Social and Governance (ESG) performance

Environmental performance

Resources

Energy consumption for operations (1,000 GJ)

Share of renewable power for production sites Water consumption for production sites (1,000 m3) Breaches of environmental regulatory limit values

Emissions and waste Scope 1 emissions (1,000 tonnes CO2) Scope 2 emissions (1,000 tonnes CO2) Scope 3 emissions (1,000 tonnes CO2)1 Waste from production sites (tonnes)

7.1

7.1

7.2

7.3

7.4

7.4

7.4

7.5

3,677

100%

3,918

75

76

16

2,041

213,505

3,387

100%

3,488

12

77

16

N/A

180,806

for the year ended 31 December

Social performance

Patients

Patients reached with Novo Nordisk's Diabetes care products (estimate in millions) – Hereof reached via 

## Finetune encoder

In [31]:
def finetune_model(model_name):
    # The next step is to fine-tune a model using MNR loss.
    # We do this easily with the sentence-transformers library.
    paths = [str(path) for path in Path('data').glob('*.tsv')]
    print(paths[:5])

    pairs = []
    for path in tqdm(paths):
        with open(path, 'r', encoding='utf-8') as fp:
            lines = fp.read().split('\n')
            for line in lines:
                if '\t' not in line:
                    continue
                else:
                    q, p = line.split('\t')
                    pairs.append(InputExample(
                        texts=[q, p]
                    ))

    # We use the no duplicates data loader to avoid placing duplicate passages in the same batch,
    # as this will confuse the ranking mechanism of MNR loss.
    batch_size = 24

    loader = datasets.NoDuplicatesDataLoader(
        pairs, batch_size=batch_size
    )

    # Now we initialize the bi-encoder that we will be fine-tuning.
    # We create the transformer-to-pooler architecture using modules.
    distilbert = models.Transformer('sentence-transformers/msmarco-distilbert-base-tas-b')
    pooler = models.Pooling(
        distilbert.get_word_embedding_dimension(),
        pooling_mode_mean_tokens=True
    )

    my_model = SentenceTransformer(modules=[distilbert, pooler])
    print(my_model)

    loss = losses.MultipleNegativesRankingLoss(my_model)

    epochs = 3
    warmup_steps = int(len(loader) * epochs * 0.1)

    my_model.fit(
        train_objectives=[(loader, loss)],
        epochs=epochs,
        warmup_steps=warmup_steps,
        output_path='my_model',
        show_progress_bar=True
    )

In [32]:
finetune_model(model_name="my_finetune_model")

['data/pairs_0.tsv', 'data/pairs_1.tsv', 'data/pairs_3.tsv', 'data/pairs_2.tsv', 'data/pairs_6.tsv']


100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 7/7 [00:00<00:00, 246.65it/s]


SentenceTransformer(
  (0): Transformer({'max_seq_length': 512, 'do_lower_case': False}) with Transformer model: DistilBertModel 
  (1): Pooling({'word_embedding_dimension': 768, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)


Epoch: 100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 3/3 [58:33<00:00, 1171.01s/it]


## Does it perform better or overfit?

In [4]:
my_embedder = SentenceTransformer("my_model")

In [5]:
encoded_query = my_embedder.encode(default_query)

In [6]:
encoded_docs = my_embedder.encode(content)

In [7]:
get_topk_similarity(
    k=10, 
    encoded_query=encoded_query, 
    encoded_docs=encoded_docs, 
    is_cos_sim=True, 
    debug=True,
)

Most similar pairs:
doc_idx	 score
260 	 0.7612
262 	 0.7410
261 	 0.7122
22 	 0.6946
19 	 0.6779
23 	 0.6744
265 	 0.6625
20 	 0.6560
21 	 0.6423
259 	 0.6351
